# preprocess liver gene expression dataset

preprocess the data from droin et al [[1]](https://www.nature.com/articles/s42255-020-00323-1) as an `anndata` object with time and spatial location (layer) `.obs` keys.

[[1] Droin, Colas, Jakob El Kholtei, Keren Bahar Halpern, Clémence Hurni, Milena Rozenberg, Sapir Muvkadi, Shalev Itzkovitz, and Felix Naef. "Space-time logic of liver gene expression at sub-lobular scale." Nature Metabolism 3, no. 1 (2021): 43-58.](https://www.nature.com/articles/s42255-020-00323-1)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import copy
import time

import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import scipy.signal
import pandas as pd
import scanpy as sc

import anndata
import re
import seaborn as sns

In [ ]:
import scvi

In [ ]:
sys.path.append("../../")
from paths import DATA_DIR

In [ ]:
DATA_DIR = str(DATA_DIR) + "/spatiotemporal_liver"

## Utility functions

In [ ]:
def parse_layer(file):
    layer_ = []
    if not os.path.exists(file):
        return -1
    df = pd.read_csv(file, delimiter=',' , header=None)
    position_= df.to_numpy()
    for i, pos in enumerate(position_):
        layer_.append(np.argmax(pos))
    return layer_

def read_sample(file, layers_path = '/content/GSE145197_LAYERS/'):
    filename_ = file.split('_')[-1].split('.')[0]
    filename_layer = layers_path +filename_ + '_reco.txt'
    print(f'parsing {filename_}')
    adata = sc.read(file).T
    adata.var_names_make_unique()
    adata.obs_names_make_unique()
    adata.obs['ZT'] = int(re.findall(r'\d+', filename_)[0])
    adata.obs['rep']  = filename_[-1]
    layers = parse_layer(filename_layer)
    adata.obs['layer'] = layers
    
    adata.obs['layer_time'] = adata.obs['layer'].astype('str') + '_' + adata.obs['ZT'].astype('str')
    
    
    adata.obs['ZT'] = adata.obs['ZT'].astype('category')
    adata.obs['layer'] = adata.obs['layer'].astype('category')
    adata.obs['layer_time'] = adata.obs['layer_time'].astype('category')
    
    return adata

def transform_data(x):
    return np.log2(x+10**-4)-np.log2(11*10**-5)


def train_scvi(adata ,layer=None, batch_key=None, continuous_covariate_keys=None):
    if layer is None:
        adata.layers["counts"] = adata.X.copy()
        layer = "counts"
    
    adata = adata.copy()
    scvi.data.setup_anndata(adata, 
                            layer=layer,
                            batch_key=batch_key,
                            continuous_covariate_keys=continuous_covariate_keys)
    
    model = scvi.model.SCVI(adata,  
                            n_hidden=128, 
                            n_layers=2,
                            gene_likelihood='nb',
                            dispersion='gene-batch'
                            )

    # MODEL TRAINING
    model.train(check_val_every_n_epoch =1, 
                    use_gpu=True,
                    plan_kwargs={'lr':1e-3})

    latent = model.get_latent_representation()
    adata.obsm['X_scvi'] = latent

    sc.pp.neighbors(adata, use_rep='X_scvi')
    sc.tl.umap(adata, min_dist=0.3)
    return adata

## Load data

download tarred files:

1. [GSE145197_RAW.tar](https://doi.org/10.6084/m9.figshare.26097655.v1)
2. [GSE145197_LAYERS.zip](https://doi.org/10.6084/m9.figshare.26097616)

### Import

In [ ]:
adatas = []
directory = DATA_DIR  + "/GSE145197_RAW"
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        adata = read_sample(f, layers_path = DATA_DIR  + "/GSE145197_LAYERS/")
        print(adata)
        if len(np.unique(adata.obs["layer"])) > 1:
            adatas.append(adata)

In [ ]:
# merge files
adata = adatas[0].concatenate(adatas[1:])

In [ ]:
adata.obs['ZT'] = adata.obs['ZT'].astype('category')
adata.obs['layer'] = adata.obs['layer'].astype('category')
adata.obs['layer_time'] = adata.obs['layer_time'].astype('category')

In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('mt-')
adata.var['mup'] = adata.var_names.str.startswith('mup')

sc.pp.calculate_qc_metrics(adata, inplace=True,qc_vars=['mt', 'mup'])

In [ ]:

sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt', 'pct_counts_mup'],
             jitter=0.4, multi_panel=True)



In [ ]:
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

In [ ]:
sc.pp.filter_genes(adata, min_cells=5)

In [ ]:
adata = adata[adata.obs.pct_counts_mt < 30, :]

In [ ]:
sc.pp.filter_cells(adata, min_genes=500)
sc.pp.filter_cells(adata, min_counts=1000)
sc.pp.filter_cells(adata, max_counts=6000)

In [ ]:
adata

In [ ]:
adata.layers["counts"] = adata.X

Normalize expression levels in each cell were normalized by the sum of all genes excluding mitochondrial and major urinary protein (_Mup_) genes.

In [ ]:
adata.X = adata.X / adata.X[:,(~adata.var['mt']) & (~adata.var['mup'])].sum(axis=1)[:, np.newaxis]

In [ ]:
adata.layers["normalized"] = adata.X

Log-transform using paper definition

In [ ]:
adata.X = transform_data(adata.X)

Filter using paper genes

In [ ]:
genes = np.loadtxt(DATA_DIR + "genes_list", dtype=str)
adata = adata[:, adata.var_names.isin(genes)]

In [ ]:
adata = adata.copy()

## Train `scVI` model

In [ ]:
batch_key="rep"

In [ ]:
adata = train_scvi(adata, 
           layer="counts", 
           batch_key=batch_key)

In [ ]:
sc.pl.umap(adata, color="ZT")

In [ ]:
sc.pl.umap(adata, color="layer")

## Save `sc` data

In [ ]:
adata.write(DATA_DIR + "adata_sc.h5ad")

## Create reference atlas

In [ ]:

def transform_data(x):
    return np.log2(x+10**-4)-np.log2(11*10**-5)

def invert_transform(y):
    return 2**(y+np.log2(11*10**-5))-10**-4 

dic_itz = {}
dic_itz_raw = {}
dic_struc = {'rep1': ['00A','06A','12A','18A'], 'rep2': ['00B','06B','12B','18B'], 'rep3': ['00C',None,'12C',None]}
for key, val in dic_struc.items():
    for x in val:
        if x is not None:
            load_path = DATA_DIR + 'raw/Profiles/ZT'+x+'.mat'
            mat = scipy.io.loadmat(load_path)
        for name, data, SD in zip(mat['all_genes'], mat['MeanGeneExp'], mat['SE']):
            if name[0][0] not in dic_itz_raw:
                dic_itz_raw[name[0][0]] = {'rep1' : np.array([]), 'rep1_std' :np.array([]), 'rep2' : np.array([]), 'rep2_std' : np.array([]), 'rep3' : np.array([]), 'rep3_std' :  np.array([])}
                dic_itz[name[0][0]] = {'rep1' : np.array([]), 'rep1_std' :np.array([]), 'rep2' : np.array([]), 'rep2_std' : np.array([]), 'rep3' : np.array([]), 'rep3_std' :  np.array([])}
            if x is None:
                data = [np.nan]*8
                SD = [np.nan]*8
            if len(dic_itz_raw[name[0][0]][key])>0:
                dic_itz_raw[name[0][0]][key] = np.vstack( (dic_itz_raw[name[0][0]][key],np.array(data) ))
                dic_itz_raw[name[0][0]][key+'_std']= np.vstack((dic_itz_raw[name[0][0]][key+'_std'],np.array(SD)))
                dic_itz[name[0][0]][key]= np.vstack( (dic_itz[name[0][0]][key],transform_data(np.array(data))))
                dic_itz[name[0][0]][key+'_std']= np.vstack( (dic_itz[name[0][0]][key+'_std'],transform_data(np.array(SD))))
            else:
                dic_itz_raw[name[0][0]][key] = np.array(data) 
                dic_itz_raw[name[0][0]][key+'_std']= np.array(SD)
                dic_itz[name[0][0]][key]= transform_data(np.array(data))
                dic_itz[name[0][0]][key+'_std']= transform_data(np.array(SD))  
        
#take transpose everywhere
for key in dic_itz:
    for key2 in ['rep1' , 'rep1_std', 'rep2', 'rep2_std', 'rep3', 'rep3_std']:
        dic_itz[key][key2] = dic_itz[key][key2].T
        dic_itz_raw[key][key2] = dic_itz_raw[key][key2].T
            
        

In [ ]:
l_circadian = ['arntl', 'clock', 'npas2', 'nr1d1', 'nr1d2', 'per1', 'per2', 'cry1', 'cry2', 'dbp', 'tef', 'hlf', 
               'elovl3', 'rora', 'rorc']
l_zonated = ['glul', 'ass1','asl','cyp2f2','cyp1a2','pck1','cyp2e1', 'cdh2','cdh1','cyp7a1','acly', 'alb', "oat", 
             "aldob", 'cps1']

Look at how the replicate variance evolves with the gene expression


In [ ]:

l_names = list(dic_itz.keys())
#compute list of variance per time condition and per zone condition and then average
l_var = np.array([ np.mean(np.nanvar([dic_itz[gene_name]['rep1'], dic_itz[gene_name]['rep2'],dic_itz[gene_name]['rep3']], axis = 0))/np.nanvar(np.vstack((dic_itz[gene_name]['rep1'], dic_itz[gene_name]['rep2'],dic_itz[gene_name]['rep3']))) for gene_name in l_names])
l_var = np.array([x if not np.isnan(x) else 10**-10 for x in l_var ])
l_exp_log = [invert_transform(np.nanmax(np.vstack((dic_itz[gene_name]['rep1'], dic_itz[gene_name]['rep2'],dic_itz[gene_name]['rep3']))))  for gene_name in l_names]
l_exp = [np.nanmax(np.vstack((dic_itz[gene_name]['rep1'], dic_itz[gene_name]['rep2'],dic_itz[gene_name]['rep3']))) for gene_name in l_names]

set_names_kept_2 = set()

#scatter plot
fig, ax = plt.subplots(figsize=(5,5))
plt.scatter(l_exp_log, l_var, s=20, alpha = 1, color = '#34495e')

#add reference genes
flag_c = True
flag_z = True
flag_u = True
for exp, var,  name in zip(l_exp_log, l_var, l_names):
    if name in l_zonated:
        if flag_z:
            plt.plot(exp, var, markersize = '20', marker = '.', lw = 0, color = "#F37F30", label = 'Reference zonated gene')
            flag_z = False
        else:
            plt.plot(exp, var, markersize = '20', marker = '.', lw = 0,color = "#F37F30")
    elif name in l_circadian:
        if flag_c:
            plt.plot(exp, var, markersize = '20', marker = '.', lw = 0,color = "#2178B4", label = 'Reference rhythmic gene')
            flag_c = False
        else:
            plt.plot(exp, var, markersize = '20', marker = '.', lw = 0,color = "#2178B4")
    if exp>10**-5 and var<0.5:
        set_names_kept_2.add(name)    
        

plt.xlim([10**-7,10**-1])
plt.ylim([0,1])
plt.xscale('log', basex=10)
plt.xlabel('Profile maximal expresion', fontsize=15)
plt.ylabel('Average relative replicates variance', fontsize=15)
plt.legend()
plt.axhline(0.5, xmin = 0.335, ls='--', color = "red", alpha = 0.8)
plt.axvline(10**-5, ymax = 0.5, ls='--', color = "red", alpha = 0.8)
plt.show()

print(len(set_names_kept_2), ' genes remaining after filtering on replicates consistency')

Look at the expresssion in the dataset

In [ ]:
#plot the histogram of expression
l_exp = [ invert_transform(np.nanmax(np.vstack((dic_itz[gene_name]['rep1'], dic_itz[gene_name]['rep2'],dic_itz[gene_name]['rep3'])))) for gene_name in dic_itz]
plt.hist(l_exp, bins=np.logspace(-8,-1, 50))
plt.xscale('log', basex=10)
plt.xlabel('Maximal expresion')
plt.show()

Filter dataset

In [ ]:
dic_itz_clean = {}
for name in set_names_kept_2:
    if 'mup' not in name and 'pisd' not in name:
        dic_itz_clean[name] = dic_itz[name]
l_names = list(dic_itz_clean.keys())

In [ ]:
current_palette = sns.color_palette('bright')
def plot_gene_name(name_gene, dic_itz):
    #plt.figure(figsize=(5,5))
    ax = plt.subplot(111)
    for t in range(4):
        plt.plot(np.linspace(1,8,8, endpoint = True), dic_itz[name_gene]['rep1'][:,t], marker=".", label = 't='+str(t*6),  color = current_palette[t], lw = 2, alpha = 0.7)
        plt.plot(np.linspace(1,8,8, endpoint = True), dic_itz[name_gene]['rep2'][:,t],  marker=".",color = current_palette[t], lw = 2,  alpha = 0.7)
        plt.plot(np.linspace(1,8,8, endpoint = True), dic_itz[name_gene]['rep3'][:,t],  marker=".",color = current_palette[t], lw = 2, alpha = 0.7)
    plt.xlabel("Layer")
    plt.ylabel("Expression")
    plt.xlim([1,8])
    plt.legend()
    #ax.legend(loc='center left')
    plt.legend(bbox_to_anchor=(1.04, 0.5), loc='center left', fontsize=14)
    plt.title(str(name_gene))
    #plt.savefig('Output/'+str(name_gene)+'.pdf', facecolor = 'white')
    plt.show()
    plt.close()
    
for name_gene in l_circadian:
    try:
        plot_gene_name(name_gene, dic_itz_clean)
    except:
        pass
for name_gene in l_zonated:
    try:
        plot_gene_name(name_gene, dic_itz_clean)
    except:
        pass

In [ ]:
load_path = DATA_DIR + 'gene_classification.csv'
gene_class_df = pd.read_csv(load_path, header=0, index_col=0, skiprows = lambda x: x in [1])


In [ ]:
F = -1
S = 0
T = 1
ST = 2
SxT = 3
nclass = 4

gene_class_name = np.asarray(['S', 'T', 'S+T', 'SxT','F'])
dic_itz_clean = {}
dic_itz_clean_raw = {}
gene_class = []
gene_class_ref = np.asarray([-1, 0, 0, 0, 1, 3, 2, 2, 2, 3, 3, 3], dtype=int)
for name in gene_class_df.T.columns:
    if 'mup' not in name and 'pisd' not in name:
        dic_itz_clean[name] = dic_itz[name]
        dic_itz_clean_raw[name] = dic_itz_raw[name]
        

l_names = np.asarray(list(dic_itz_clean.keys()))
gene_class = np.asarray([gene_class_ref[np.argmax(gene_class_df.T[name])] for name in l_names])
gene_prob = np.asarray([gene_class_df.T[name] for name in l_names])
gene_confidence = np.asarray([np.max(gene_class_df.T[name]) for name in l_names])



In [ ]:
gene_prob

In [ ]:
types = ["ZT", "layer", "layer_time"]

In [ ]:
dfs = {}
for type_ in types:
    dfs[type_] = pd.DataFrame(index=adata.obs[type_].cat.categories)

In [ ]:
rep1 = 'rep1'
rep2 = 'rep2'
for gi, gene in enumerate(l_names):
    gene_atlas = (dic_itz_clean[gene][rep1] + dic_itz_clean[gene][rep2]) /2
    
    dfs[types[-1]] = pd.concat([dfs[types[-1]] , 
                                pd.DataFrame(gene_atlas.flatten(), 
                                             columns=[gene], 
                                             index=adata.obs[types[-1]].cat.categories)],
                               axis=1,
                              )

    for i, type_ in enumerate(types[:-1]):
        dfs[type_] =  pd.concat([dfs[type_], 
                                 pd.DataFrame(np.mean(gene_atlas, axis=i), 
                                              columns=[gene], index=adata.obs[type_].cat.categories)],
                                axis=1,

                               )

In [ ]:
adatas = {}
for type_ in types:
    adatas[type_] = anndata.AnnData(dfs[type_])
    adatas[type_].var["type"] = gene_class
    adatas[type_].var["gene_confidence"] = gene_confidence
    adatas[type_].varm["type_prob"] = gene_prob
    

In [ ]:
theta = np.arange(0, 2 * np.pi, 2 * np.pi/len(adata.obs["ZT"].cat.categories.values))

x = np.cos(theta)
y =  np.sin(theta)
t_locations = np.array(list(zip(x, y)))

z = adata.obs["layer"].cat.categories.values
st_locations = np.array([(zi, x[j], y[j]) for zi in z for j in range(len(adata.obs["ZT"].cat.categories.values))])

locations = [t_locations, z, st_locations]

In [ ]:
for i, type_ in enumerate(types):
    adatas[type_].obsm["spatial"] = locations[i]
    adatas[type_].obsm["spatial"] = adatas[type_].obsm["spatial"].astype(float)

In [ ]:
for key, adata_ in adatas.items():
    adata_.write(DATA_DIR + f'adata_{key}.h5ad')